In [ ]:
!pip install transformers datasets

In [ ]:
import re
import torch
from datasets import load_dataset, DatasetDict
from transformers import BertForQuestionAnswering, BertTokenizer, AutoTokenizer, pipeline
#from google.colab import drive
#drive.mount('/content/gdrive')

## Load Data

In [ ]:
train = load_dataset('json', data_files='data/train.json')['train']
dev = load_dataset('json', data_files='data/dev.json')['train']
test = load_dataset('json', data_files='data/test.json')['train']

data = DatasetDict({'train': train, 'dev': dev})
data

## Preprocessing

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

test = ['he is losing it, it', '#endangeredriver, #endangereddriver']
inputs = tokenizer(
        test,
        max_length=128,
        truncation="only_second",
        padding="max_length",
    )
type(inputs)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

def preprocess_function(examples):
    questions = [q.strip() for q in examples["Question"]]
    inputs = tokenizer(
        questions,
        examples["Tweet"],
        max_length=128,
        truncation="only_second",
        padding="max_length",
    )

    answers = [q[0].strip() for q in examples["Answer"]]
    answer_tokens = tokenizer(
        answers,
        max_length=128,
        truncation="only_second",
        padding="max_length",
    )

    #input['answer_input_ids'] = answers
    #input['token_type_ids'] = "answer_tokens['token_type_ids']"
    #input['attention_mask'] = "answer_tokens['attention_mask']"
    print(inputs)
    return inputs
  
tokenized_data = data.map(preprocess_function, batched=True, remove_columns=data["train"].column_names)

In [ ]:
tokenized_data